<a href="https://colab.research.google.com/github/luishernand/Analisis-_EDA_predictions/blob/master/finance_porfolio_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/luishernand/pandas_fundamentals/blob/master/logo4.JPG?raw=true" >  

**Instructor:** [Computer Science](http://everythingcomputerscience.com/)  
**Descripción:** optimizador de cartera

|Email|Fecha|
|-----|-----|
|luishernandezmatos@yahoo.com|25 de mayo 2020|

In [0]:
from IPython.display import YouTubeVideo
YouTubeVideo('9fjs8FeLMJk')

# Finance Portfolio Optimization

La optimización de la cartera es el proceso de seleccionar la mejor cartera ( distribución de activos ), fuera del conjunto de todas las carteras consideradas, de acuerdo con algún objetivo. El objetivo generalmente maximiza factores como el rendimiento esperado y minimiza costos como el riesgo financiero . Los factores que se consideran pueden variar desde tangibles (como activos , pasivos , ganancias u otros fundamentos ) hasta intangibles (como desinversiones selectivas )

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from datetime import datetime
from pandas_datareader import data as web
from IPython.display import YouTubeVideo

In [0]:
assets = ['FB', 'AMZN', 'AAPL', 'NFLX', 'GOOG']
#asignar pesos a las carteras
weights = np.array([0.2,0.2,0.2,0.2,0.2])
stock_startdate = '2013-01-01'
hoy = datetime.today().strftime('%Y-%m-%d')

### Cargar los datos

In [0]:
df = pd.DataFrame()
for stock in assets:
 df[stock] = web.DataReader(stock, data_source='yahoo', start=stock_startdate ,
                            end = hoy)['Adj Close']

In [0]:
df.head()

In [0]:
df.tail()

### Visualizar los datos

In [0]:
df.plot(figsize=(12.5, 4.5))
plt.title('Portfolio Adj close history price', size = 20)
plt.ylabel('Precio de cierre en USD$', size = 12)
plt.show()

### Correlaciones y evaluación de riesgos 
---  

Diferentes enfoques para la optimización de la cartera miden el riesgo de manera diferente. Además de la medida tradicional, la desviación estándar o su cuadrado ( varianza ), que no son medidas de riesgo sólidas , otras medidas incluyen la relación de Sortino , CVaR (valor condicional en riesgo) y la dispersión estadística .  

La inversión es una actividad prospectiva y, por lo tanto, las covarianzas de los rendimientos deben pronosticarse en lugar de observarse.  

La optimización de la cartera supone que el inversor puede tener cierta aversión al riesgo y que los precios de las acciones pueden presentar diferencias significativas entre sus valores históricos o pronosticados y lo experimentado. En particular, las crisis financieras se caracterizan por un aumento significativo en la correlación de los movimientos del precio de las acciones que pueden degradar seriamente los beneficios de la diversificación.    

En un marco de optimización de varianza media, la estimación precisa de la matriz de varianza-covarianza es primordial. Las técnicas cuantitativas que utilizan la simulación de Montecarlo con la cópula gaussiana y las distribuciones marginales bien especificadas son efectivas.  Permitir que el proceso de modelado permita características empíricas en los rendimientos de las existencias, como la autorregresión , la volatilidad asimétrica, la asimetría y la curtosis es importante. No tener en cuenta estos atributos puede conducir a un error grave de estimación en las correlaciones, variaciones y covarianzas que tienen sesgos negativos (hasta un 70% de los valores verdaderos).   

Otras estrategias de optimización que se centran en minimizar el riesgo de cola (por ejemplo, valor en riesgo , valor condicional en riesgo ) en las carteras de inversión son populares entre los inversores adversos al riesgo. Para minimizar la exposición al riesgo de cola, los pronósticos de rendimiento de activos utilizando la simulación de Montecarlo con cópulas de vid para permitir una menor dependencia de cola (izquierda) (por ejemplo, Clayton, Gumbel rotado) en grandes carteras de activos son los más adecuados.   

Más recientemente, los administradores de fondos de cobertura han estado aplicando una "optimización a gran escala" mediante la cual cualquier función de utilidad del inversor se puede utilizar para optimizar una cartera.  Se afirma que dicha metodología es más práctica y adecuada para los inversores modernos cuyas preferencias de riesgo implican reducir el riesgo de cola , minimizando la asimetría negativa y las colas gruesas en la distribución de los rendimientos de la cartera de inversiones. Cuando dichas metodologías implican el uso de funciones de utilidad de momento superior, es necesario utilizar una metodología que permita pronosticar una distribución conjuntaeso explica la dependencia asimétrica. Una metodología adecuada que permite que la distribución conjunta incorpore dependencia asimétrica es la Clayula Canonical Vine Copula. Ver Cópula (teoría de la probabilidad) # Finanzas cuantitativas .  

### Cooperación en optimización de cartera   

Un grupo de inversores, en lugar de invertir individualmente, puede optar por invertir su capital total en la cartera conjunta, y luego dividir el beneficio de inversión (incierto) de la manera que mejor se adapte a sus preferencias de utilidad / riesgo. Resulta que, al menos en el modelo de utilidad esperado,  y el modelo de desviación media,  cada inversionista generalmente puede obtener una participación que valora estrictamente más que su cartera óptima de la inversión individual.

---
### Análisis_de forma manual

In [0]:
#retorno diario simple
returns = df.pct_change()
returns

In [0]:
# matriz de covarianza anual
cov_mtx_anual = returns.cov()*252
cov_mtx_anual

In [0]:
# calculo de la cartera_varianza
port_var = np.dot(weights.T, np.dot(cov_mtx_anual, weights))
port_var

In [0]:
#calculo de la volatividad o la desviación estadar
port_volatibidad = np.sqrt(port_var)
port_volatibidad

In [0]:
#retorno annual de cartera
port_annual_returns = np.sum(returns.mean()* weights)*252
port_annual_returns

In [0]:
# ver la expectativa de retorno, volatividad o riesgo y la varianza
percent_var = str(round(port_var,2)*100)+ '%'
percent_vol = str(round(port_volatibidad,2)*100)+ '%'
percent_return = str(round(port_annual_returns,2)*100)+ '%'

In [0]:
print('expected annual return: ' + percent_return)
print('expected volatividad/riesgo: ' + percent_vol)
print('expected varianza: ' + percent_var)

### Análisis (ML)

ahora haremos los análisis realizados anteriormente pero utilizando **machine learning**

In [0]:
pip install PyPortfolioOpt

In [0]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [0]:
#calulo de las covarianza, del retorno y de la volatividad
mu = expected_returns.mean_historical_return(df)
s = risk_models.sample_cov(df)

In [0]:
#optimizador max shape ratio
ef = EfficientFrontier(mu,s)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [0]:
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

---
#### Calcular los Fondos restantes y su asignación discreta de $15,000

In [0]:
from pypfopt.discrete_allocation import  DiscreteAllocation, get_latest_prices

In [0]:
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=15000)

In [0]:
allocation, leftover = da.lp_portfolio()

In [0]:
print('Asignación discreta:', allocation)
print('fondos restantes: ${:.2f}'.format(leftover))